In [16]:
# #import .env variables
# from dotenv import dotenv_values
# config = dotenv_values(".env")
# config["USERNAME"]

'ssoundarajan@calmi2.org'

In [17]:
#imports
import boto3
import requests
import pandas as pd 
from tqdm import tqdm

In [18]:
# def authenticate_user():
#     PENNSIEVE_URL = "https://api.pennsieve.io"
#     email = config["USERNAME"]
#     password = config["PASSWORD"]

#     r = requests.get(f"{PENNSIEVE_URL}/authentication/cognito-config")
#     r.raise_for_status()
#     cognito_app_client_id = r.json()["userPool"]["appClientId"]
#     cognito_region = r.json()["userPool"]["region"]

#     cognito_client = boto3.client(
#     "cognito-idp",
#     region_name=cognito_region,
#     aws_access_key_id="",
#     aws_secret_access_key="")
    
#     login_response = cognito_client.initiate_auth(
#     AuthFlow="USER_PASSWORD_AUTH",
#     AuthParameters={"USERNAME": email, "PASSWORD": password},
#     ClientId=cognito_app_client_id)
    
#     api_key = login_response["AuthenticationResult"]["AccessToken"]
    
#     r = requests.get(f"{PENNSIEVE_URL}/user", headers={"Authorization": f"Bearer {api_key}"})
#     r.raise_for_status()
#     print(r.json())
#     return api_key 
    
# api_key = authenticate_user()

{'id': 'N:user:65290a77-3d9e-4b43-9500-60a4a26ebc89', 'email': 'ssoundarajan@calmi2.org', 'firstName': 'Sanjay', 'lastName': 'Soundarajan', 'credential': 'Software Developer', 'color': '#474647', 'url': '', 'authyId': 0, 'isSuperAdmin': False, 'createdAt': '2020-12-07T23:37:58.087015Z', 'updatedAt': '2021-07-01T17:28:21.852925Z', 'preferredOrganization': 'N:organization:618e8dd9-f8d2-4dc4-9abb-c6aaab2e78a0', 'orcid': {'name': 'Sanjay Soundarajan', 'orcid': '0000-0003-2829-8032'}, 'pennsieveTermsOfService': {'version': '20210710000000'}, 'customTermsOfService': [], 'intId': 1079}


In [32]:
def get_published_datasets():
    url = "https://api.pennsieve.io/discover/search/records"
    querystring = {"model": "award"}
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()
    querystring = {"limit": response["totalCount"], "model": "award"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()
    list_of_datasets = response["records"].copy()
    return list_of_datasets

list_of_datasets = get_published_datasets()
list_of_datasets[0]

for item in tqdm(list_of_datasets):
    url = f"https://api.pennsieve.io/discover/datasets/{item['datasetId']}"
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers)
    response = response.json()
    item['version'] = response['version']
    item['versionPublishedAt'] = response['versionPublishedAt']
    item['datasetDOI'] = 'https://dx.doi.org/' + response['doi']
    item['tags'] = response['tags']
    item['contributors'] = response['contributors']

100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:35<00:00,  2.76it/s]


In [33]:
#change loop variant on prod
# for item in list_of_datasets[:1]:
#     url = f"https://api.pennsieve.io/discover/datasets/{item['datasetId']}/versions/{item['version']}/files"
#     querystring = {"path":"files/dataset_description.xlsx"}
#     headers = {"Accept": "application/json"}
#     response = requests.request("GET", url, headers=headers, params=querystring)
#     response = response.json()
#     item["datasetDescriptionPackageId"] = response["sourcePackageId"]
#     print(response)

In [34]:
#change loop variant on prod
# loop_variant = list_of_datasets[:2]
loop_variant = list_of_datasets

for item in tqdm(loop_variant):
    url = "https://api.pennsieve.io/zipit/discover"    
    payload = {"data": {
        "paths": ["files/dataset_description.xlsx"],
        "version": item['version'],
        "datasetId": item['datasetId']
    }}
    response = requests.request("POST", url, json=payload,)
    local_filename = "dataset_description.xlsx"
    totalbits = 0
    if response.status_code == 200:
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    totalbits += 1024
#                     print("Downloaded",totalbits*1025,"KB...")
                    f.write(chunk)
        try: 
            xl = pd.ExcelFile("dataset_description.xlsx")
            df = xl.parse("Sheet1")

            # arrays to hold dois for each dataset
            originating_article_array = []
            protocol_array = []

            # get all dois from dataframe
            for index, row in df.iterrows():
                row_val = row['Metadata element']
                if (row_val.find('Originating Article') != -1):
                    count = 0
                    for col_val in row:
                        if (count > 2):
                            try: 
                                if col_val.find('doi.org') != -1:
                                    pos = col_val.find('http')
                                    if pos != -1:
                                        col_val = col_val[pos:]
                                    originating_article_array.append(col_val)
                            except:
                                pass
                        count = count + 1
                if (row_val.find('Protocol') != -1):
                    count = 0
                    for col_val in row:
                        if (count > 2):
                            try: 
                                if (col_val.find('doi.org') != -1):
                                    pos = col_val.find('http')
                                    if pos != -1:
                                        col_val = col_val[pos:]
                                    protocol_array.append(col_val)
                            except:
                                pass
                        count = count + 1
            item["originatingArticleDOI"] = originating_article_array
            item["protocolsDOI"] = protocol_array
        except: 
            print(item)
            item["originatingArticleDOI"] = []
            item["protocolsDOI"] = []

for item in loop_variant:
    print( item["originatingArticleDOI"],  item["protocolsDOI"])

 12%|██████████                                                                        | 12/98 [00:14<01:42,  1.20s/it]

{'datasetId': 137, 'version': 1, 'model': 'award', 'properties': {'description': '', 'id': 'feebb68c-e847-4a9e-9931-e7f1e5332a57', 'principal_investigator': '', 'title': '', 'award_id': 'OT2OD023848'}, 'versionPublishedAt': '2021-05-11T19:37:19.02855Z', 'datasetDOI': 'https://dx.doi.org/10.26275/56h4-ypua', 'tags': ['sparc', 'ragp', 'lcm', 'single neurons', '3d tracking'], 'contributors': [{'firstName': 'Alison ', 'middleInitial': None, 'lastName': 'Moss', 'degree': None, 'orcid': '0000-0002-7907-8796'}, {'firstName': 'Shaina', 'middleInitial': None, 'lastName': 'Robbins', 'degree': None, 'orcid': '0000-0002-7885-743X'}, {'firstName': 'Sirisha', 'middleInitial': None, 'lastName': 'Achanta', 'degree': None, 'orcid': None}, {'firstName': 'Sean', 'middleInitial': None, 'lastName': 'Nieves', 'degree': None, 'orcid': None}, {'firstName': 'Scott', 'middleInitial': None, 'lastName': 'Turick', 'degree': None, 'orcid': None}, {'firstName': 'Peter', 'middleInitial': None, 'lastName': 'Hanna', 'd

 44%|███████████████████████████████████▉                                              | 43/98 [01:00<01:04,  1.17s/it]

{'datasetId': 74, 'version': 2, 'model': 'award', 'properties': {'description': 'DESCRIPTION. A high-density Stim-Grid MAP spinal cord stimulation lead technology is proposed to enablethe creation of predictive maps that detail how lower urinary tract nerve pathways can be activated through high-density stimulation of the sacral spinal cord and roots. These maps will enable principled development ofneuromodulation therapies that use spinal cord stimulation to alleviate the burden of overactive bladder andincontinence. The lower urinary tract (LUT), consisting of the bladder, urethra and associated muscles and nerves,is an important target organ system for neuromodulation therapies as injury, disease and aging can lead toimpairment and subsequent reductions in the quality of life. While existing neuromodulation devices have beenimplanted in over 200,000 people, device related adverse events remain common, and despite functionalimprovements, many people still deal with unwanted symptoms 

 94%|████████████████████████████████████████████████████████████████████████████▉     | 92/98 [02:12<00:07,  1.30s/it]

{'datasetId': 137, 'version': 1, 'model': 'award', 'properties': {'description': '', 'id': 'feebb68c-e847-4a9e-9931-e7f1e5332a57', 'principal_investigator': '', 'title': '', 'award_id': 'OT2OD023848'}, 'versionPublishedAt': '2021-05-11T19:37:19.02855Z', 'datasetDOI': 'https://dx.doi.org/10.26275/56h4-ypua', 'tags': ['sparc', 'ragp', 'lcm', 'single neurons', '3d tracking'], 'contributors': [{'firstName': 'Alison ', 'middleInitial': None, 'lastName': 'Moss', 'degree': None, 'orcid': '0000-0002-7907-8796'}, {'firstName': 'Shaina', 'middleInitial': None, 'lastName': 'Robbins', 'degree': None, 'orcid': '0000-0002-7885-743X'}, {'firstName': 'Sirisha', 'middleInitial': None, 'lastName': 'Achanta', 'degree': None, 'orcid': None}, {'firstName': 'Sean', 'middleInitial': None, 'lastName': 'Nieves', 'degree': None, 'orcid': None}, {'firstName': 'Scott', 'middleInitial': None, 'lastName': 'Turick', 'degree': None, 'orcid': None}, {'firstName': 'Peter', 'middleInitial': None, 'lastName': 'Hanna', 'd

100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [02:22<00:00,  1.45s/it]

[] ['dx.doi.org/10.17504/protocols.io.6bvhan6']
[] ['dx.doi.org/10.17504/protocols.io.bafnibme']
['https://doi.org/10.1111/nmo.13380'] []
[] []
[] ['https://dx.doi.org/10.17504/protocols.io.wqmfdu6']
[] ['dx.doi.org/10.17504/protocols.io.bh4dj8s6']
[] []
['https://doi.org/10.3389/fphys.2018.00785'] []
[] []
[] []
[] []
[] []
[] []
['https://doi.org/10.1371/journal.pcbi.1002061'] []
[] ['dx.doi.org/10.17504/protocols.io.5nkg5cw']
[] ['https://dx.doi.org/10.17504/protocols.io.9gbh3sn', 'https://dx.doi.org/10.17504/protocols.io.y3tfynn', 'https://dx.doi.org/10.17504/protocols.io.bjfzkjp6']
[] []
[] []
[] ['https://dx.doi.org/10.17504/protocols.io.wjrfcm6']
[] []
[] []
['https://doi.org/10.1007/s00441-018-2957-0'] []
['https://doi.org/10.1088/1741-2552/ab7ad4'] []
[] ['https://doi.org/10.17504/protocols.io.2irgcd6']
['https://doi.org/10.1038/s41598-021-81822-3'] ['dx.doi.org/10.17504/protocols.io.w6hfhb6']
[] []
['https://doi.org/10.1113/jp273259'] []
[] ['dx.doi.org/10.17504/protocols.io.

In [35]:
for item in loop_variant:
    print(item["datasetId"], item["version"], item["originatingArticleDOI"],  item["protocolsDOI"])

64 4 [] ['dx.doi.org/10.17504/protocols.io.6bvhan6']
121 1 [] ['dx.doi.org/10.17504/protocols.io.bafnibme']
24 2 ['https://doi.org/10.1111/nmo.13380'] []
31 3 [] []
54 1 [] ['https://dx.doi.org/10.17504/protocols.io.wqmfdu6']
65 6 [] ['dx.doi.org/10.17504/protocols.io.bh4dj8s6']
27 2 [] []
35 3 ['https://doi.org/10.3389/fphys.2018.00785'] []
58 1 [] []
55 1 [] []
127 1 [] []
137 1 [] []
103 4 [] []
63 2 ['https://doi.org/10.1371/journal.pcbi.1002061'] []
23 2 [] ['dx.doi.org/10.17504/protocols.io.5nkg5cw']
43 5 [] ['https://dx.doi.org/10.17504/protocols.io.9gbh3sn', 'https://dx.doi.org/10.17504/protocols.io.y3tfynn', 'https://dx.doi.org/10.17504/protocols.io.bjfzkjp6']
29 6 [] []
119 1 [] []
56 1 [] ['https://dx.doi.org/10.17504/protocols.io.wjrfcm6']
76 3 [] []
10 3 [] []
59 1 ['https://doi.org/10.1007/s00441-018-2957-0'] []
97 1 ['https://doi.org/10.1088/1741-2552/ab7ad4'] []
123 1 [] ['https://doi.org/10.17504/protocols.io.2irgcd6']
130 1 ['https://doi.org/10.1038/s41598-021-81822-3